In [4]:
#import data
import json
import pandas as pd
file_name_1 = "Segment prediction/raw data/train_data.json"
with open(file_name_1, 'r') as jsonfile1:
    data_dict_1 = json.load(jsonfile1)
train = pd.DataFrame.from_dict(data_dict_1, orient='index')
del data_dict_1
train.reset_index(level=0, inplace=True)
train.rename(columns = {'index':'ID'},inplace=True)
train.shape

(200000, 7)

In [7]:
print(list(train))

['ID', 'tod', 'genres', 'titles', 'cities', 'segment', 'dow']


In [4]:
print(train.head())

            ID segment                                        cities  \
0      train-1     neg                     gurgaon:55494,delhi:31892   
1     train-10     neg             delhi:5862,nagar:8916,mumbai:1593   
2    train-100     neg                              navi mumbai:4142   
3   train-1000     neg  new delhi:4131,chennai:2878,navi mumbai:1339   
4  train-10000     neg                     gurgaon:6077,chennai:4055   

                                                 tod  \
0  10:26,13:331,12:323,20:21864,21:16233,17:7953,...   
1  11:1661,10:384,20:401,21:798,22:221,16:525,19:...   
2                                1:1207,0:2406,2:529   
3        11:71,20:2417,21:1042,23:2288,19:1872,8:658   
4  20:158,22:4139,17:67,23:1510,19:288,18:56,0:23...   

                                   genres  \
0  Cricket:82379,Kabaddi:255,Reality:4751   
1              Cricket:15640,Wildlife:730   
2                 LiveTV:13,Football:4129   
3               TalkShow:658,Cricket:7690   
4    

In [8]:
def dict_converter(x):
    if x== "":
        return {}
    
    lst = []
    for m in x.split(","):
        if m =='':
            continue
        f,y = m.rsplit(":",1)
        lst.append((f,int(y)))
    
    return dict(lst)

def dow_labeller(x):
    k = []
    for i in x.split(","):
        i = "dow"+i 
        k.append(i)
    k = ",".join(k)
    return k

def tod_labeller(x):
    k = []
    for i in x.split(","):
        i = "tod"+i 
        k.append(i)
    k = ",".join(k)
    return k

Now time to do the vectoriseing

In [9]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
vec = DictVectorizer()

train['dow_encoded'] = train['dow'].apply(dow_labeller)
train['tod_encoded'] = train['tod'].apply(tod_labeller)

#Todo need to combine all the colums and add a special unique label for dow and tod
features_str= train[['genres','dow_encoded', 'tod_encoded', 'titles','cities']].\
    apply(lambda x: ','.join(x), axis=1)

#target_train = train["segment"]

#Encode Target Variable
target_train = le.fit_transform(train["segment"])
#now we delete the original data from memory
del train
#convert to dicts
dict_features  = features_str.apply(dict_converter)

sparce_matrix = vec.fit_transform(dict_features)
#remove the dict features because a sparce matrix now exists
del dict_features
#print(sparce_matrix)

In [31]:
#scaling all the information
#from sklearn.preprocessing import StandardScaler
#sparce_matrix = StandardScaler(with_mean=False).fit_transform(sparce_matrix)
#sparce_matrix

In [78]:
#using the pipeling
from time import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing


#label_encoder = preprocessing.LabelEncoder()
#encoded_target_y_train= label_encoder.fit_transform(y_train)

parameters = {'min_samples_split':[120,400,800]}
#clf = RandomForestClassifier(oob_score=True,n_estimators=30,max_features =2)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
grid = GridSearchCV(clf, parameters,scoring='roc_auc')

t0 = time()
grid.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0)) 
#print(sorted(clf.cv_results_.keys()))
#print(clf.cv_results_)
print("The best parameters are %s with a score of %0.5f"
      % (grid.best_params_, grid.best_score_))

In [10]:
# Utility function to report best scores
import numpy as np
from time import gmtime, strftime
def report(results, n_top=10):
    print("Report generated at {}".format(strftime("%d-%m-%Y %H:%M:%S")))
    print("")
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f}),average time:{2:.3f}".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate],
                results["mean_fit_time"][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [16]:
from time import time
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.externals import joblib
from time import strftime
import numpy as np

# build a classifier
clf = XGBClassifier()


# specify parameters and distributions to sample from

param_dist = {"learning_rate":sp_uniform(0.01,0.4),
              #"gamma":sp_uniform(0.001,30),
              "n_estimators":sp_randint(150,220),
              "max_depth":sp_randint(3,8),
              "colsample_bytree":sp_uniform(0.3,0.5),
               'subsample': np.random.uniform(0.5,1,1000),
               'reg_lambda': sp_uniform(0.3,1.5)
}

# run randomized search
n_iter_search = 100
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='roc_auc',cv=None
                                   )

start = time()
random_search.fit(sparce_matrix, target_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

# activate this when fitting models
current_time = strftime("%d-%b-%Y %H-%M-%S")
joblib.dump(random_search, 'Segment prediction/classifier pickle dump/'
                           'segment prediction_{0} pv score {1:.6f}.pkl'.format(current_time,
                                                                        random_search.best_score_))

report(random_search.cv_results_)

RandomizedSearchCV took 11932.62 seconds for 100 candidates parameter settings.
Report generated at 28-06-2017 17:18:11



Model with rank: 1
Mean validation score: 0.827025 (std: 0.004220),average time:35.593
Parameters: {'learning_rate': 0.074328918328445445, 'reg_lambda': 0.42678029949742707, 'subsample': 0.96846932037563438, 'max_depth': 7, 'colsample_bytree': 0.35670493653765878, 'n_estimators': 170}

Model with rank: 2
Mean validation score: 0.826928 (std: 0.005382),average time:43.444
Parameters: {'learning_rate': 0.145907868143172, 'reg_lambda': 1.6759095377781985, 'subsample': 0.81878581631716152, 'max_depth': 5, 'colsample_bytree': 0.72438232769269661, 'n_estimators': 182}

Model with rank: 3
Mean validation score: 0.826834 (std: 0.004380),average time:34.186
Parameters: {'learning_rate': 0.1105041056296911, 'reg_lambda': 1.0693131482316618, 'subsample': 0.76491427861680505, 'max_depth': 6, 'colsample_bytree': 0.56182934177578192, 'n_estimators': 155}

Model with rank: 4
Mean validation score: 0.826822 (std: 0.004998),average time:43.402
Parameters: {'learning_rate': 0.11399894858281655, 'reg_l

In [19]:
report(random_search.cv_results_)

Report generated at 22-06-2017 00:03:51

Model with rank: 1
Mean validation score: 0.827363 (std: 0.004511),average time:46.605
Parameters: {'reg_lambda': 1.1554720354390606, 'n_estimators': 202, 'colsample_bytree': 0.71198994591581355, 'subsample': 0.7232702377618665, 'max_depth': 6, 'learning_rate': 0.11801404268080393}

Model with rank: 2
Mean validation score: 0.823433 (std: 0.004854),average time:23.699
Parameters: {'reg_lambda': 0.8214060039127089, 'n_estimators': 125, 'colsample_bytree': 0.70377985227615791, 'subsample': 0.99173723170900918, 'max_depth': 4, 'learning_rate': 0.098782826890588052}

Model with rank: 3
Mean validation score: 0.822420 (std: 0.003790),average time:15.952
Parameters: {'reg_lambda': 0.57704789716848337, 'n_estimators': 93, 'colsample_bytree': 0.46030457478925302, 'subsample': 0.75079320699186769, 'max_depth': 5, 'learning_rate': 0.28759071515500184}

Model with rank: 4
Mean validation score: 0.820113 (std: 0.004900),average time:15.343
Parameters: {'reg

In [20]:
params =random_search.cv_results_["params"]
scores = random_search.cv_results_["mean_test_score"]
params_df = pd.DataFrame(data=list(zip(scores,params)),
                         columns=["scores","params"])
params_df.sort_values(by="scores",inplace=True)
loc = -2
a = params_df.iloc[loc]["params"]
print("params",a)
best_score =  100+params_df.iloc[loc]["scores"]
print("best score",best_score)
special_clf = XGBClassifier(**a)
special_clf.fit(sparce_matrix, target_train)

params {'learning_rate': 0.145907868143172, 'reg_lambda': 1.6759095377781985, 'subsample': 0.81878581631716152, 'max_depth': 5, 'colsample_bytree': 0.72438232769269661, 'n_estimators': 182}
best score 100.826928412


XGBClassifier(base_score=0.5, colsample_bylevel=1,
       colsample_bytree=0.72438232769269661, gamma=0,
       learning_rate=0.145907868143172, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=182, nthread=-1,
       objective='binary:logistic', reg_alpha=0,
       reg_lambda=1.6759095377781985, scale_pos_weight=1, seed=0,
       silent=True, subsample=0.81878581631716152)

#Now time to submit the data


In [12]:
import json
import pandas as pd
#submitting stuff
file_name_2 = "Segment prediction/raw data/test_data.json"
with open(file_name_2, 'r') as jsonfile2:
    data_dict_2 = json.load(jsonfile2)

test = pd.DataFrame.from_dict(data_dict_2, orient='index')
del data_dict_2
test.reset_index(level=0, inplace=True)
test.rename(columns = {'index':'ID'},inplace=True)
submission_df = pd.DataFrame(data = test["ID"])

test.shape

(100000, 6)

In [4]:
list(test)

['ID', 'dow', 'genres', 'cities', 'titles', 'tod']

In [13]:
#convert to sparce matrix test

test['dow_encoded'] = test['dow'].apply(dow_labeller)
test['tod_encoded'] = test['tod'].apply(tod_labeller)

#Todo need to combine all the colums and add a special unique label for dow and tod
features_str_test= test[['genres','dow_encoded', 'tod_encoded', 'titles','cities']].\
    apply(lambda x: ','.join(x), axis=1)

del test
dict_features_test  = features_str_test.apply(dict_converter)
del features_str_test

sparce_matrix_test = vec.transform(dict_features_test)
del dict_features_test

In [22]:
#can skip to this if the test data is loaded and model is trained
import numpy as np
from time import strftime

# View probabilities
probs_test = random_search.predict_proba(sparce_matrix_test)
#probs_test = special_clf.predict_proba(sparce_matrix_test)

#del sparce_matrix_test
submission_df["segment"] = probs_test[:,1]


current_time = strftime("%d-%b-%Y %H-%M-%S")
submission_df.to_csv("Segment prediction/output_data/segment_prediction_output_{0}"
                     " pv score {1:.6f}.csv".format(current_time,random_search.best_score_)
                     ,index = False)